In [1]:
import copy
import numpy as np
import os
import verdict

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
import matplotlib.patheffects as path_effects
import matplotlib.gridspec as gridspec
import matplotlib.transforms as transforms

In [3]:
import analysis_config

In [4]:
import linefinder.analyze_data.worldlines as a_worldlines
import linefinder.analyze_data.worldline_set as a_w_set
import linefinder.analyze_data.plot_worldlines as p_worldlines

In [5]:
import linefinder.utils.presentation_constants as p_constants
import linefinder.utils.file_management as file_management
import linefinder.config as linefinder_config

In [6]:
import galaxy_dive.plot_data.plotting as plotting
import galaxy_dive.analyze_data.particle_data as particle_data
import galaxy_dive.utils.astro as astro_utils
import galaxy_dive.utils.utilities as utilities
import galaxy_dive.utils.executable_helpers as exec_helpers
import galaxy_dive.utils.data_operations as data_operations
import galaxy_dive.plot_data.qual_colormaps as qual_colormaps

# Load Data

In [7]:
do_calculation = True
do_calc_of_ang_momentum = False
calc_full_dist = False

In [8]:
snum, galdef = exec_helpers.choose_config_or_commandline(
    [ analysis_config.SNUM, analysis_config.GALDEF ]
)
print( 'Using snum {}, galdef {}'.format( snum, galdef ) )

Using snum 172, galdef 


In [9]:
tag_tail = '_CGM_snum{}'.format( snum )

In [10]:
ahf_index = 600

In [11]:
ind = ahf_index - snum

In [12]:
# Load the a helper for loading files easily
file_manager = file_management.FileManager( project='CGM_fate' )

In [13]:
defaults, variations = file_manager.get_linefinder_analysis_defaults_and_variations(
    tag_tail, 
    sim_names = analysis_config.SIM_NAMES,
    galdef = galdef,
)

In [14]:
w_set = a_w_set.WorldlineSet( defaults, variations )

# Analyze Data

## Setup

In [15]:
classification_list = copy.copy( p_constants.CLASSIFICATIONS_CGM_FATE )

In [16]:
w_set.data_object.retrieve_halo_data()
halo_masses = w_set.data_object.m_vir.inner_item( snum )

Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


In [17]:
z_min = halo_masses.array().min()/1.5
z_max = halo_masses.array().max()*1.5

## Full Temperature Profiles

In [23]:
bins = np.logspace( 0., 90., 256 )

In [24]:
if calc_full_dist:

    # Start from a fresh slate
    w_set.data_object.data_masker.clear_masks( True )
    # Choose only gas
    w_set.data_object.data_masker.mask_data( 'PType', data_value=linefinder_config.PTYPE_GAS )

In [25]:
if calc_full_dist:
    
    for i, r_start in enumerate( radial_bins[:-1] ):
        w_set.data_object.data_masker.mask_data(
            'R',
            r_start,
            radial_bins[i+1],
            scale_key = 'Rvir',
            scale_a_power = 1.,
            scale_h_power = -1.,
            optional_mask = True,
            mask_name = r_start,
        )
    w_set.data_object.data_masker.mask_data(
        'T',
        0.,
        10.**4.7,
        optional_mask = True,
        mask_name = 'cool',
    )

    w_set.data_object.data_masker.mask_data(
        'T',
        10.**4.7,
        np.inf,
        optional_mask = True,
        mask_name = 'warm-hot',
    )

In [26]:
if calc_full_dist:

    hists = {}

    for version in [ 1, 2, ]:

        fig = plt.figure( figsize=(10,16), facecolor='white' )
        main_ax = plt.gca()

        gs = matplotlib.gridspec.GridSpec( len( classification_list ), 1, )

        gs.update(wspace=0.025, hspace=0.0001)

        y_max = 0
        for i, classification in enumerate( classification_list ):

            hists[classification] = verdict.Dict( {} )
            hists[classification]['cuts'] = verdict.Dict( {} )

            ax = plt.subplot( gs[i,0] )

            for sim_name, w_plotter in w_set.items():

                # Plot only some simulations
                if version == 1:
                    if sim_name not in [ 'm10y', 'm11v', 'm12i', ]:
                        continue

                # Get the data
                w = w_plotter.data_object
                w.calc_abs_phi( normal_vector=tot_momentums[sim_name] )
                
                # Only plot when there's enough data
                n_class = w.get_selected_data( classification, sl=(slice(None),ind), ).sum()
                if n_class < 1000:
                    print( 'Insufficient {} for sim {}'.format( classification, sim_name ) )
                    continue

                # Color
                m_vir = halo_masses[sim_name]
                z_width = np.log10( z_max ) - np.log10( z_min )
                color_value = ( np.log10( m_vir ) - np.log10( z_min ) )/z_width
                color = cm.viridis( color_value )

                hist, edges = w_plotter.histogram(
                    'AbsPhi',
                    bins = bins,
                    weight_key = 'M',
                    ax = ax,
                    mask_zeros = True,
                    norm_type = 'probability',
                    smooth = True,
                    smoothing_window_length = 31,
                    histogram_style = 'line',
                    x_label = r'$\Phi$ $(\degree)$',
                    y_label = p_constants.CLASSIFICATION_LABELS[classification],
                #     bins = used_bins,
                    slices = ind,
                    plot_label = None,
                    color = color,
                    assert_contains_all_data = False,
                    classification = classification,
                    return_dist = True,
                )
                
                # Track the maxes so we can set plot limits
                if hist.max() > y_max:
                    y_max = hist.max()

                # Store the data
                if version == 2:
                    hists[classification][sim_name] = hist
                    
                if version == 2:
                    hists[classification]['cuts'][sim_name] = {}

                    for T_mask in [ None, 'cool', 'warm-hot' ]:

                        hists[classification]['cuts'][sim_name][T_mask] = {}

                        r_masks = list( copy.deepcopy( radial_bins[:-1] ) )
                        r_masks.append( None )
                        for r_mask in r_masks:

                            optional_masks = []
                            if T_mask is not None:
                                optional_masks.append( T_mask )
                            if r_mask is not None:
                                optional_masks.append( r_mask )

                            hist, edges = w_plotter.histogram(
                                'AbsPhi',
                                bins = bins,
                                weight_key = 'M',
                                ax = ax,
                                mask_zeros = True,
                                norm_type = 'probability',
                                smooth = True,
                                smoothing_window_length = 31,
                                histogram_style = 'line',
                                x_label = r'$\Phi$ $(\degree)$',
                                y_label = p_constants.CLASSIFICATION_LABELS[classification],
                            #     bins = used_bins,
                                slices = ind,
                                plot_label = None,
                                color = color,
                                assert_contains_all_data = False,
                                classification = classification,
                                return_dist = True,
                                optional_masks = optional_masks
                            )

                            hists[classification]['cuts'][sim_name][T_mask][r_mask] = hist

                # Try and plot a spherical distribution
                phi_a = np.linspace( 0., 90., 1024 )
                y_spherical = pdf_spherical( phi_a )
                ax.plot(
                    phi_a,
                    y_spherical,
                    color = 'k',
                    linestyle = '--',
                    linewidth = 3,
                )

            # Adjust tick parameters
            ax.tick_params( direction='inout', which='both', top=True, )

            if classification != 'is_in_CGM':
                ax.yaxis.label.set_path_effects(
                    [
                        path_effects.Stroke(
                            linewidth=2,
                            foreground=p_constants.CLASSIFICATION_COLORS_B[classification]
            #                 foreground='k',
                        ),
                        path_effects.Normal() 
                    ]
                )

            # Add another x-axis label
            if i == 0:
                ax.annotate(
                    s = r'$\Phi$ $(\degree)$',
                    xy = ( 0.5, 1.17, ),
                    xycoords = 'axes fraction',
                    fontsize = 22,
                    ha = 'center',
                )

            # Add a label to the y axes
            if i == 0:

                formatted_redshift = '{:.02g}'.format( w.redshift.values[ind] )

                ax.annotate(
    #                 s = r'$\frac{d(M/M_{\rm tot})}{d(r/r_{\rm vir})}' + r'(z={})$'.format( formatted_redshift ),
                    s = r'PDF $' + r'(z={})$'.format( formatted_redshift ),
                    xy = ( 0, 1.2, ),
                    xycoords = 'axes fraction',
                    fontsize = 28,
                    ha = 'center',
                )

            # Hide labels
            if i != len( classification_list ) - 1:
                ax.tick_params( labelbottom = False )
            if i == 0:
                ax.tick_params( labeltop = True )

            # Avoid overlapping ticks
            ax.get_yticklabels()[0].set_verticalalignment( 'bottom' )
            ax.get_yticklabels()[-1].set_verticalalignment( 'top' )
        #     ax.set_yticklabels([0., 0.5, 1. ], va='top' )

        # Set limits
        for i, classification in enumerate( classification_list ):
            ax = plt.subplot( gs[i,0] )
            ax.set_xlim( 0., 90. )
            ax.set_ylim( 0., 1.05*y_max )

        # Add a colorbar
        sm = cm.ScalarMappable(
            cmap = cm.viridis,
            norm=colors.LogNorm( z_min, z_max ),
        )
        sm._A = []
        plotting.add_colorbar(
            fig,
            sm,
            method='fig',
            ax_location=[0.905, 0.125, 0.03, 0.76]
        )
        plt.subplot( gs[0,0] ).annotate(
            s = r'$M_{\rm h}$ $(M_{\odot})$',
            xy = (1., 1.2),
        #     xy = (1.15, 0.5),
            xycoords = 'axes fraction',
            fontsize = 22,
            ha = 'center',
        )

    #     if version == 1:
    #         save_file = 'CGM_profile_snum{}.pdf'.format( snum )

    #         plotting.save_fig(
    #             out_dir = file_manager.get_project_figure_dir(),
    #             save_file = save_file,
    #             fig = fig,
    #         )

In [27]:
if calc_full_dist:
    # Transpose to get the simulation name on the lowest level
    hists = verdict.Dict( hists )
    for classification in classification_list:
        hists[classification]['cuts'] = hists[classification]['cuts'].transpose()
        for key, item in hists[classification]['cuts'].items():
            hists[classification]['cuts'][key] = item.transpose()

In [28]:
savefile = os.path.join(
    file_manager.project_parameters['output_data_dir'],
    'azimuthal_angle_distributions_snum{}.hdf5'.format( snum ),
)

In [29]:
if calc_full_dist:
    
    hists.to_hdf5(
        savefile, 
        condensed = False, 
    )

In [30]:
if not calc_full_dist:
    
    hists = verdict.Dict.from_hdf5( savefile, )

OSError: Unable to open file (unable to open file: name = '/work/03057/zhafen/stampede2/nbs/CGM_fate_analysis/data/azimuthal_angle_distributions_snum172.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

## Plot Aggregate

In [ ]:
n_rows = len( classification_list )
n_cols = len( radial_bins[:-1] )

medians = {}

T_cuts = [ None, 'cool', 'warm-hot' ]
T_labels = [ 'All gas', r'$T < 10^{4.7}$ K', r'$T > 10^{4.7}$ K' ]

fig = plt.figure( figsize=(n_rows*3.5,n_cols*3.5), facecolor='white' )
main_ax = plt.gca()

gs = matplotlib.gridspec.GridSpec( n_rows, n_cols, )

gs.update(wspace=0.0001, hspace=0.0001)

for i, classification in enumerate( classification_list ):

    for j, T_cut in enumerate( T_cuts ):

        medians[classification] = verdict.Dict( {} )

        y_max = 0

        try:
            class_hists = hists[classification]['cuts'][T_cut][None].split_by_dict( linefinder_config.MASS_BINS )['m12']
        except KeyError:
            class_hists = hists[classification]['cuts'][str(T_cut)][str(None)].split_by_dict( linefinder_config.MASS_BINS )['m12']

        sub_arr = class_hists.array() / pdf_spherical( bins[:-1] )

        ax = plt.subplot( gs[i,j] )

        median = np.nanmedian( sub_arr, axis=0 )
        ax.plot(
            bins[:-1],
            median,
            color = 'k',
            linewidth = 3,
        )

        ax.fill_between(
            bins[:-1],
            np.nanmin( sub_arr, axis=0 ),
            np.nanmax( sub_arr, axis=0 ),
            color = p_constants.CLASSIFICATION_COLORS_B[classification],
            alpha = 0.3,
        )

        ax.fill_between(
            bins[:-1],
            np.nanpercentile( sub_arr, 16, axis=0 ),
            np.nanpercentile( sub_arr, 84, axis=0 ),
    #         np.nanpercentile( sub_arr, 50 - 68/2, axis=0 ),
    #         np.nanpercentile( sub_arr, 50 + 68/2, axis=0 ),        
            color = p_constants.CLASSIFICATION_COLORS_B[classification],
            alpha = 0.4,
        )

        # Try and plot a spherical distribution
        phi_a = np.linspace( 0., 90., 1024 )
        y_spherical = pdf_spherical( bins[:-1] )
        ax.plot(
            bins[:-1],
            np.ones( bins[:-1].shape ),
            color = 'k',
            linestyle = '--',
            linewidth = 3,
        )

        # Track the maxes so we can set plot limits
        hist_max = np.nanmax( class_hists.array() )

        if hist_max > y_max:
            y_max = hist_max

        ax.set_xlabel(
            r'Polar Angle, $\Phi$ $(\degree)$',
            fontsize = 22,
        )
        if ax.is_first_col():
            ax.set_ylabel(
                p_constants.CLASSIFICATION_LABELS[classification],
                fontsize = 22,
            )

        ax.set_xlim( 0., 90. )
        ax.set_ylim( 0., 3., )
        ax.set_yscale( 'linear' )

        # Adjust tick parameters
        ax.tick_params( direction='inout', which='both', top=True, )

        # Rotate labels
    #     ax.yaxis.label.set_rotation( 'horizontal' )
    #     ax.yaxis.label.set_color(
    #         p_constants.CLASSIFICATION_COLORS_B[classification]
    #     )

#             if classification != 'is_in_CGM':
#                 ax.yaxis.label.set_path_effects(
#                     [
#                         path_effects.Stroke(
#                             linewidth=2,
#                             foreground=p_constants.CLASSIFICATION_COLORS_B[classification]
#             #                 foreground='k',
#                         ),
#                         path_effects.Normal() 
#                     ]
#                 )

        # Add another x-axis label
        if i == 0:

            ax.annotate(
                s = T_labels[j],
                xy = ( 0.5, 1.25, ),
                xycoords = 'axes fraction',
                fontsize = 22,
                ha = 'center',
            )

        if i == 2:
            ax.legend(
                prop={'size': 18},
                loc = 'upper right',
            )

        # Add a label to the y axes
        if i == 0 and ax.is_first_col():

            formatted_redshift = '{:.02g}'.format( redshift )

            ax.annotate(
                s = r'$M_{{\rm h}} \sim 10^{12} M_\odot$, $' + r'z={}$'.format( formatted_redshift ),
                xy = ( -0.25, 1.45, ),
                xycoords = 'axes fraction',
                fontsize = 24,
                ha = 'left',
            )
            
            ax.annotate(
                s = r'$M(\Phi$ | origin ) / $M(\Phi)_{{\rm spherical}}$',
                xy = ( -0.25, -1., ),
                xycoords = 'axes fraction',
                fontsize = 24,
                ha = 'right',
                va = 'center',
                rotation = 90,
            )

        # Hide labels
        if i != len( classification_list ) - 1:
            ax.tick_params( labelbottom = False )
        if i == 0:
            ax.tick_params( labeltop = True )

        if not ax.is_first_col():
            ax.tick_params( labelleft = False )
        else:
            # Avoid overlapping ticks
            ax.get_yticklabels()[0].set_verticalalignment( 'bottom' )
            ax.get_yticklabels()[-1].set_verticalalignment( 'top' )
        #     ax.set_yticklabels([0., 0.5, 1. ], va='top' )
    
#         # Avoid overlapping labels
#         ax.get_xticklabels()[0].set_horizontalalignment( 'left' )
#         ax.get_xticklabels()[-1].set_horizontalalignment( 'right' )

save_file = 'CGM_polar_dists_m12s_snum{}.pdf'.format( snum )
# plotting.save_fig(
#     out_dir = file_manager.get_project_figure_dir(),
#     save_file = save_file,
#     fig = fig,
# )

# Analysis of the Plot

In [ ]:
import py2tex.py2tex as py2tex

In [ ]:
tex_filepath = os.path.join( file_manager.project_parameters['project_dir'], 'variables.tex' )

In [ ]:
tex_file = py2tex.TeXVariableFile( tex_filepath )